In [2]:
# Function to parse input into a grid format
def parse_input(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

# Function to find all numbers and their positions in the grid
def find_numbers_and_positions(grid):
    numbers_map = {}
    for i, row in enumerate(grid):
        j = 0
        while j < len(row):
            if row[j].isdigit():
                number = ""
                start_j = j
                while j < len(row) and row[j].isdigit():
                    number += row[j]
                    j += 1
                if number not in numbers_map:
                    numbers_map[number] = []
                numbers_map[number].append((i, start_j, j - 1))  # Store start and end column indices
            else:
                j += 1
    return numbers_map

# Load the sample input file
sample_input_path = 'sample-input.txt'
sample_input = parse_input(sample_input_path)

# Find numbers and their positions in the sample input
sample_numbers_map = find_numbers_and_positions(sample_input)

import pandas as pd
# Convert to DataFrame for better display
df = pd.DataFrame([{"Number": k, "Positions": v} for k, v in sample_numbers_map.items()])

df

,Number,Positions
0,467,"[(0, 0, 2)]"
1,114,"[(0, 5, 7)]"
2,35,"[(2, 2, 3)]"
3,633,"[(2, 6, 8)]"
4,617,"[(4, 0, 2)]"
5,58,"[(5, 7, 8)]"
6,592,"[(6, 2, 4)]"
7,755,"[(7, 6, 8)]"
8,664,"[(9, 1, 3)]"
9,598,"[(9, 5, 7)]"


In [6]:
# Function to find positions of stars and their bounding boxes in the grid
def find_stars_and_bounding_boxes(grid):
    stars_map = {}
    star_count = 1

    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if cell == '*':
                # Star position
                star_position = (i, j)
                
                # Bounding box positions
                bounding_positions = []
                for dx in range(-1, 2):  # Rows in the bounding box
                    for dy in range(-1, 2):  # Columns in the bounding box
                        nx, ny = i + dx, j + dy
                        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and (nx, ny) != (i, j):
                            bounding_positions.append((nx, ny))
                
                # Add to map with unique star identifier
                stars_map[f"Star {star_count}"] = {
                    "Star Position": star_position,
                    "Bounding Positions": bounding_positions
                }
                star_count += 1

    return stars_map

# Find stars and their bounding boxes in the sample input
stars_map = find_stars_and_bounding_boxes(sample_input)

# Convert to DataFrame for better display
df_stars = pd.DataFrame([{"Star": k, **v} for k, v in stars_map.items()])

df_stars

,Star,Star Position,Bounding Positions
0,Star 1,"(1, 3)","[(0, 2), (0, 3), (0, 4), (1, 2), (1, 4), (2, 2..."
1,Star 2,"(4, 3)","[(3, 2), (3, 3), (3, 4), (4, 2), (4, 4), (5, 2..."
2,Star 3,"(8, 5)","[(7, 4), (7, 5), (7, 6), (8, 4), (8, 6), (9, 4..."


In [7]:
# Function to map each position to the number it belongs to
def map_positions_to_numbers(grid):
    position_to_number_map = {}

    for i, row in enumerate(grid):
        j = 0
        while j < len(row):
            if row[j].isdigit():
                number = ""
                start_j = j
                while j < len(row) and row[j].isdigit():
                    number += row[j]
                    j += 1
                # Map all positions of this number to the number
                for pos in range(start_j, j):
                    position_to_number_map[(i, pos)] = number
            else:
                j += 1
    return position_to_number_map

# Create position-to-number map for the sample input
position_to_number_map = map_positions_to_numbers(sample_input)

# Convert to DataFrame for better display
df_positions = pd.DataFrame(
    [{"Position": k, "Number": v} for k, v in position_to_number_map.items()]
)

df_positions

,Position,Number
0,"(0, 0)",467
1,"(0, 1)",467
2,"(0, 2)",467
3,"(0, 5)",114
4,"(0, 6)",114
5,"(0, 7)",114
6,"(2, 2)",35
7,"(2, 3)",35
8,"(2, 6)",633
9,"(2, 7)",633


In [8]:
# Function to find shared positions between stars and numbers
def find_shared_positions(grid, stars_map, position_to_number_map):
    shared_positions = []
    
    for star_info in stars_map.values():
        for position in star_info["Bounding Positions"]:
            if position in position_to_number_map:
                shared_positions.append({
                    "Star Position": star_info["Star Position"],
                    "Shared Position": position,
                    "Number": position_to_number_map[position]
                })
    
    return shared_positions

# Find shared positions in the sample input
shared_positions = find_shared_positions(sample_input, stars_map, position_to_number_map)

# Convert to DataFrame for better display
df_shared_positions = pd.DataFrame(shared_positions)

df_shared_positions

,Star Position,Shared Position,Number
0,"(1, 3)","(0, 2)",467
1,"(1, 3)","(2, 2)",35
2,"(1, 3)","(2, 3)",35
3,"(4, 3)","(4, 2)",617
4,"(8, 5)","(7, 6)",755
5,"(8, 5)","(9, 5)",598
6,"(8, 5)","(9, 6)",598


In [9]:
# Function to group stars with exactly two adjacent numbers
def group_stars_with_two_numbers(grid, stars_map, position_to_number_map):
    star_groups = []
    
    for star_key, star_info in stars_map.items():
        adjacent_numbers = {}
        
        for position in star_info["Bounding Positions"]:
            if position in position_to_number_map:
                number = position_to_number_map[position]
                if number not in adjacent_numbers:
                    adjacent_numbers[number] = []
                adjacent_numbers[number].append(position)
        
        # Check if the star has exactly two unique adjacent numbers
        if len(adjacent_numbers) == 2:
            numbers = list(adjacent_numbers.keys())
            product = int(numbers[0]) * int(numbers[1])
            star_groups.append({
                "Star": star_key,
                "Star Position": star_info["Star Position"],
                "First Number": numbers[0],
                "Second Number": numbers[1],
                "Product": product
            })
    
    return star_groups

# Group stars with two adjacent numbers in the sample input
star_groups = group_stars_with_two_numbers(sample_input, stars_map, position_to_number_map)

# Convert to DataFrame for better display
df_star_groups = pd.DataFrame(star_groups)

df_star_groups

,Star,Star Position,First Number,Second Number,Product
0,Star 1,"(1, 3)",467,35,16345
1,Star 3,"(8, 5)",755,598,451490


In [10]:
# Calculate the sum of the products
total_product_sum = sum(group["Product"] for group in star_groups)

total_product_sum

467835